In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
import catboost as cb

# Загрузка данных
data = pd.read_csv("heart.csv", usecols=["cp", "chol", "restecg", "target"])
data.dropna(inplace=True)

# Вывод первых нескольких строк данных
print(data.head())

# Проверка наличия пропущенных значений
print(data.isnull().sum())

# Определение признаков и целевой переменной
predicts = data[['cp', 'chol', 'restecg']]
target = data['target']

# Вывод признаков и целевой переменной
print(predicts)
print(target)
print(predicts.shape)

# Разделение на обучающую и тестовую выборки
A_train, A_test, y_train, y_test = train_test_split(predicts, target, train_size=0.8)
print(A_train.shape)
print(A_test.shape)

# Обучение случайного леса с регуляризацией
random_forest = RandomForestClassifier(max_depth=15, min_samples_split=10, max_features='sqrt').fit(A_train, y_train)
y_preds_d = random_forest.predict(A_train)

print('F1 мера для тренировочных данных:', f1_score(y_preds_d, y_train, average='macro'))
y_pred = random_forest.predict(A_test)
print('F1 мера для тестовых данных:', f1_score(y_pred, y_test, average='macro'))

# Подбор оптимальных параметров для случайного леса
random_forest = RandomForestClassifier()
params_grid = {
    "max_depth": [12, 18],
    "min_samples_leaf": [3, 10],
    "min_samples_split": [6, 12],
}
grid_search_random_forest = GridSearchCV(estimator=random_forest, param_grid=params_grid, scoring='f1_macro', cv=4)
grid_search_random_forest.fit(A_train, y_train)
best_model = grid_search_random_forest.best_estimator_
y_preds_d = best_model.predict(A_train)

print('F1 мера для тренировочных данных:', f1_score(y_preds_d, y_train, average='macro'))
y_pred = best_model.predict(A_test)
print('F1 мера для тестовых данных:', f1_score(y_pred, y_test, average='macro'))

# Обучение модели CatBoost с регуляризацией
model_catboost_clf = cb.CatBoostClassifier(iterations=3000, task_type='GPU', devices='0', l2_leaf_reg=3)
model_catboost_clf.fit(A_train, y_train)
y_preds_t = model_catboost_clf.predict(A_train, task_type='CPU')

print('F1 мера для тренировочных данных:', f1_score(y_preds_t, y_train, average='macro'))
y_preds = model_catboost_clf.predict(A_test, task_type='CPU')
print('F1 мера для тестовых данных:', f1_score(y_preds, y_test, average='macro'))

   cp  chol  restecg  target
0   3   233        0       1
1   2   250        1       1
2   1   204        0       1
3   1   236        1       1
4   0   354        1       1
cp         0
chol       0
restecg    0
target     0
dtype: int64
     cp  chol  restecg
0     3   233        0
1     2   250        1
2     1   204        0
3     1   236        1
4     0   354        1
..   ..   ...      ...
298   0   241        1
299   3   264        1
300   0   193        1
301   0   131        1
302   1   236        0

[303 rows x 3 columns]
0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64
(303, 3)
(242, 3)
(61, 3)
F1 мера для тренировочных данных: 0.8095987411487018
F1 мера для тестовых данных: 0.751696065128901
F1 мера для тренировочных данных: 0.7457983193277311
F1 мера для тестовых данных: 0.7689393939393939
0:	learn: 0.6841074	total: 39ms	remaining: 1m 57s
1:	learn: 0.6733486	total: 89.8ms	remaining: 

145:	learn: 0.4434529	total: 6.39s	remaining: 2m 4s
146:	learn: 0.4433148	total: 6.43s	remaining: 2m 4s
147:	learn: 0.4424235	total: 6.48s	remaining: 2m 4s
148:	learn: 0.4423812	total: 6.51s	remaining: 2m 4s
149:	learn: 0.4423153	total: 6.55s	remaining: 2m 4s
150:	learn: 0.4417447	total: 6.61s	remaining: 2m 4s
151:	learn: 0.4411970	total: 6.66s	remaining: 2m 4s
152:	learn: 0.4409789	total: 6.71s	remaining: 2m 4s
153:	learn: 0.4406116	total: 6.75s	remaining: 2m 4s
154:	learn: 0.4398250	total: 6.8s	remaining: 2m 4s
155:	learn: 0.4396998	total: 6.86s	remaining: 2m 5s
156:	learn: 0.4390109	total: 6.91s	remaining: 2m 5s
157:	learn: 0.4388432	total: 6.96s	remaining: 2m 5s
158:	learn: 0.4388201	total: 7s	remaining: 2m 5s
159:	learn: 0.4382199	total: 7.05s	remaining: 2m 5s
160:	learn: 0.4377538	total: 7.09s	remaining: 2m 4s
161:	learn: 0.4368219	total: 7.14s	remaining: 2m 5s
162:	learn: 0.4359932	total: 7.19s	remaining: 2m 5s
163:	learn: 0.4359720	total: 7.22s	remaining: 2m 4s
164:	learn: 0.43

306:	learn: 0.3948697	total: 14.1s	remaining: 2m 3s
307:	learn: 0.3948186	total: 14.1s	remaining: 2m 3s
308:	learn: 0.3944155	total: 14.2s	remaining: 2m 3s
309:	learn: 0.3943842	total: 14.2s	remaining: 2m 3s
310:	learn: 0.3941475	total: 14.3s	remaining: 2m 3s
311:	learn: 0.3941387	total: 14.3s	remaining: 2m 3s
312:	learn: 0.3940850	total: 14.3s	remaining: 2m 3s
313:	learn: 0.3939386	total: 14.4s	remaining: 2m 3s
314:	learn: 0.3934013	total: 14.4s	remaining: 2m 3s
315:	learn: 0.3932041	total: 14.5s	remaining: 2m 3s
316:	learn: 0.3931737	total: 14.5s	remaining: 2m 3s
317:	learn: 0.3931664	total: 14.6s	remaining: 2m 2s
318:	learn: 0.3931124	total: 14.6s	remaining: 2m 2s
319:	learn: 0.3930639	total: 14.7s	remaining: 2m 2s
320:	learn: 0.3923359	total: 14.7s	remaining: 2m 2s
321:	learn: 0.3917990	total: 14.8s	remaining: 2m 2s
322:	learn: 0.3917374	total: 14.8s	remaining: 2m 2s
323:	learn: 0.3912946	total: 14.8s	remaining: 2m 2s
324:	learn: 0.3910149	total: 14.9s	remaining: 2m 2s
325:	learn: 

467:	learn: 0.3633898	total: 21.5s	remaining: 1m 56s
468:	learn: 0.3631621	total: 21.5s	remaining: 1m 56s
469:	learn: 0.3631596	total: 21.6s	remaining: 1m 56s
470:	learn: 0.3631321	total: 21.6s	remaining: 1m 55s
471:	learn: 0.3631300	total: 21.6s	remaining: 1m 55s
472:	learn: 0.3629987	total: 21.7s	remaining: 1m 55s
473:	learn: 0.3627543	total: 21.7s	remaining: 1m 55s
474:	learn: 0.3627523	total: 21.8s	remaining: 1m 55s
475:	learn: 0.3626254	total: 21.8s	remaining: 1m 55s
476:	learn: 0.3626226	total: 21.9s	remaining: 1m 55s
477:	learn: 0.3625788	total: 21.9s	remaining: 1m 55s
478:	learn: 0.3625764	total: 22s	remaining: 1m 55s
479:	learn: 0.3622488	total: 22s	remaining: 1m 55s
480:	learn: 0.3620952	total: 22.1s	remaining: 1m 55s
481:	learn: 0.3620928	total: 22.1s	remaining: 1m 55s
482:	learn: 0.3620112	total: 22.1s	remaining: 1m 55s
483:	learn: 0.3615949	total: 22.2s	remaining: 1m 55s
484:	learn: 0.3613924	total: 22.2s	remaining: 1m 55s
485:	learn: 0.3612378	total: 22.3s	remaining: 1m 5

623:	learn: 0.3375848	total: 29.6s	remaining: 1m 52s
624:	learn: 0.3372051	total: 29.6s	remaining: 1m 52s
625:	learn: 0.3367479	total: 29.7s	remaining: 1m 52s
626:	learn: 0.3357316	total: 29.7s	remaining: 1m 52s
627:	learn: 0.3356863	total: 29.8s	remaining: 1m 52s
628:	learn: 0.3356244	total: 29.8s	remaining: 1m 52s
629:	learn: 0.3349933	total: 29.9s	remaining: 1m 52s
630:	learn: 0.3349493	total: 29.9s	remaining: 1m 52s
631:	learn: 0.3348338	total: 30s	remaining: 1m 52s
632:	learn: 0.3347910	total: 30s	remaining: 1m 52s
633:	learn: 0.3346855	total: 30.1s	remaining: 1m 52s
634:	learn: 0.3346345	total: 30.2s	remaining: 1m 52s
635:	learn: 0.3345927	total: 30.2s	remaining: 1m 52s
636:	learn: 0.3345530	total: 30.3s	remaining: 1m 52s
637:	learn: 0.3344976	total: 30.3s	remaining: 1m 52s
638:	learn: 0.3343664	total: 30.4s	remaining: 1m 52s
639:	learn: 0.3343432	total: 30.4s	remaining: 1m 52s
640:	learn: 0.3343206	total: 30.5s	remaining: 1m 52s
641:	learn: 0.3342829	total: 30.5s	remaining: 1m 5

...

2311:	learn: 0.2408522	total: 2m 1s	remaining: 36.1s
2312:	learn: 0.2408269	total: 2m 1s	remaining: 36.1s
2313:	learn: 0.2408236	total: 2m 1s	remaining: 36s
2314:	learn: 0.2408150	total: 2m 1s	remaining: 36s
2315:	learn: 0.2408019	total: 2m 1s	remaining: 35.9s
2316:	learn: 0.2407616	total: 2m 1s	remaining: 35.9s
2317:	learn: 0.2407172	total: 2m 1s	remaining: 35.8s
2318:	learn: 0.2407149	total: 2m 1s	remaining: 35.8s
2319:	learn: 0.2407082	total: 2m 1s	remaining: 35.7s
2320:	learn: 0.2406846	total: 2m 1s	remaining: 35.7s
2321:	learn: 0.2406746	total: 2m 1s	remaining: 35.6s
2322:	learn: 0.2406461	total: 2m 2s	remaining: 35.6s
2323:	learn: 0.2406377	total: 2m 2s	remaining: 35.5s
2324:	learn: 0.2405948	total: 2m 2s	remaining: 35.5s
2325:	learn: 0.2405877	total: 2m 2s	remaining: 35.4s
2326:	learn: 0.2405781	total: 2m 2s	remaining: 35.3s
2327:	learn: 0.2405597	total: 2m 2s	remaining: 35.3s
2328:	learn: 0.2405590	total: 2m 2s	remaining: 35.2s
2329:	learn: 0.2405571	total: 2m 2s	remaining: 35.

2469:	learn: 0.2379152	total: 2m 10s	remaining: 28s
2470:	learn: 0.2377651	total: 2m 10s	remaining: 27.9s
2471:	learn: 0.2377535	total: 2m 10s	remaining: 27.9s
2472:	learn: 0.2377397	total: 2m 10s	remaining: 27.8s
2473:	learn: 0.2377383	total: 2m 10s	remaining: 27.8s
2474:	learn: 0.2377322	total: 2m 10s	remaining: 27.7s
2475:	learn: 0.2377257	total: 2m 10s	remaining: 27.7s
2476:	learn: 0.2377252	total: 2m 10s	remaining: 27.6s
2477:	learn: 0.2377124	total: 2m 10s	remaining: 27.5s
2478:	learn: 0.2377103	total: 2m 10s	remaining: 27.5s
2479:	learn: 0.2377099	total: 2m 10s	remaining: 27.4s
2480:	learn: 0.2377076	total: 2m 10s	remaining: 27.4s
2481:	learn: 0.2376738	total: 2m 10s	remaining: 27.3s
2482:	learn: 0.2376725	total: 2m 11s	remaining: 27.3s
2483:	learn: 0.2376711	total: 2m 11s	remaining: 27.2s
2484:	learn: 0.2376669	total: 2m 11s	remaining: 27.2s
2485:	learn: 0.2376650	total: 2m 11s	remaining: 27.1s
2486:	learn: 0.2376453	total: 2m 11s	remaining: 27.1s
2487:	learn: 0.2376433	total: 

2625:	learn: 0.2352897	total: 2m 18s	remaining: 19.8s
2626:	learn: 0.2352718	total: 2m 18s	remaining: 19.7s
2627:	learn: 0.2352712	total: 2m 18s	remaining: 19.7s
2628:	learn: 0.2352271	total: 2m 19s	remaining: 19.6s
2629:	learn: 0.2352266	total: 2m 19s	remaining: 19.6s
2630:	learn: 0.2351974	total: 2m 19s	remaining: 19.5s
2631:	learn: 0.2351971	total: 2m 19s	remaining: 19.5s
2632:	learn: 0.2351841	total: 2m 19s	remaining: 19.4s
2633:	learn: 0.2351796	total: 2m 19s	remaining: 19.4s
2634:	learn: 0.2351778	total: 2m 19s	remaining: 19.3s
2635:	learn: 0.2351244	total: 2m 19s	remaining: 19.3s
2636:	learn: 0.2351225	total: 2m 19s	remaining: 19.2s
2637:	learn: 0.2351211	total: 2m 19s	remaining: 19.2s
2638:	learn: 0.2351102	total: 2m 19s	remaining: 19.1s
2639:	learn: 0.2351031	total: 2m 19s	remaining: 19.1s
2640:	learn: 0.2351027	total: 2m 19s	remaining: 19s
2641:	learn: 0.2351023	total: 2m 19s	remaining: 19s
2642:	learn: 0.2350916	total: 2m 19s	remaining: 18.9s
2643:	learn: 0.2350603	total: 2m

2780:	learn: 0.2328210	total: 2m 27s	remaining: 11.6s
2781:	learn: 0.2328138	total: 2m 27s	remaining: 11.6s
2782:	learn: 0.2328062	total: 2m 27s	remaining: 11.5s
2783:	learn: 0.2328052	total: 2m 27s	remaining: 11.5s
2784:	learn: 0.2328038	total: 2m 27s	remaining: 11.4s
2785:	learn: 0.2327964	total: 2m 27s	remaining: 11.4s
2786:	learn: 0.2327854	total: 2m 27s	remaining: 11.3s
2787:	learn: 0.2327851	total: 2m 27s	remaining: 11.2s
2788:	learn: 0.2327784	total: 2m 27s	remaining: 11.2s
2789:	learn: 0.2327749	total: 2m 28s	remaining: 11.1s
2790:	learn: 0.2327687	total: 2m 28s	remaining: 11.1s
2791:	learn: 0.2327671	total: 2m 28s	remaining: 11s
2792:	learn: 0.2327585	total: 2m 28s	remaining: 11s
2793:	learn: 0.2327575	total: 2m 28s	remaining: 10.9s
2794:	learn: 0.2327320	total: 2m 28s	remaining: 10.9s
2795:	learn: 0.2327242	total: 2m 28s	remaining: 10.8s
2796:	learn: 0.2327232	total: 2m 28s	remaining: 10.8s
2797:	learn: 0.2327216	total: 2m 28s	remaining: 10.7s
2798:	learn: 0.2327071	total: 2m

2933:	learn: 0.2299258	total: 2m 36s	remaining: 3.52s
2934:	learn: 0.2299254	total: 2m 36s	remaining: 3.47s
2935:	learn: 0.2299227	total: 2m 36s	remaining: 3.41s
2936:	learn: 0.2299202	total: 2m 36s	remaining: 3.36s
2937:	learn: 0.2299199	total: 2m 36s	remaining: 3.31s
2938:	learn: 0.2299167	total: 2m 36s	remaining: 3.25s
2939:	learn: 0.2299159	total: 2m 36s	remaining: 3.2s
2940:	learn: 0.2299155	total: 2m 36s	remaining: 3.15s
2941:	learn: 0.2299061	total: 2m 36s	remaining: 3.09s
2942:	learn: 0.2299010	total: 2m 36s	remaining: 3.04s
2943:	learn: 0.2299005	total: 2m 36s	remaining: 2.99s
2944:	learn: 0.2299000	total: 2m 37s	remaining: 2.93s
2945:	learn: 0.2298808	total: 2m 37s	remaining: 2.88s
2946:	learn: 0.2298728	total: 2m 37s	remaining: 2.83s
2947:	learn: 0.2298470	total: 2m 37s	remaining: 2.77s
2948:	learn: 0.2298446	total: 2m 37s	remaining: 2.72s
2949:	learn: 0.2297682	total: 2m 37s	remaining: 2.67s
2950:	learn: 0.2297471	total: 2m 37s	remaining: 2.61s
2951:	learn: 0.2297444	total: